<center> <h1>QOSF Task 1 Tackling: generative modeling</h1> </center>
\\
<center> <h3>Mohamed Sami ShehabEldin</h3> </center>
<center> <h4>Zewail City of Science and Technology</h4> </center>

**Note**: this is just the workshop for the task, the work may be not detailed. it will be full complete and detailed by the next week (deadline of task submission)

### Task Statement:

![title](images/c1.png)

<img src="images/c2.png" width="600" height="100">

<img src="images/c3.png" width="600" height="100">

### Key Functions:

In [21]:
from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.quantum_info import random_statevector
from scipy import optimize

Function to generate odd blocks:

In [8]:
def odd_block(n=4,th_i=[None],block_name="U_i",par_gate="rz"):
    pii=np.pi
    if th_i[0]==None:
        th_i=np.round(np.random.rand(n)*2*pii,3)
        
    qc=QuantumCircuit(n)
    for i in range(n):
        if par_gate=="rz":
            qc.rz(th_i[i],i)
        elif par_gate=="rx":
            qc.rx(th_i[i],i)
        elif par_gate=="ry":
            qc.ry(th_i[i],i)
        elif par_gate=="u1":
            qc.u1(th_i[i],i)
        else:
            raise Exception("the gate should be paramaterized by single parameter!")
            
    for i in range(4):
        for j in range(i+1,4):
            qc.cz(i,j)
        
            
    qc.name = block_name
    return qc

In [9]:
#example
odd_block().draw()

┌───────────┐                  
q_0: ┤ RZ(5.636) ├─■──■──■──────────
     ├───────────┤ │  │  │          
q_1: ┤ RZ(3.081) ├─■──┼──┼──■──■────
     ├───────────┤    │  │  │  │    
q_2: ┤ RZ(4.275) ├────■──┼──■──┼──■─
     ├───────────┤       │     │  │ 
q_3: ┤ RZ(3.959) ├───────■─────■──■─
     └───────────┘

Function to generate even blocks:

In [10]:
def even_block(n=4,th_i=[None],block_name="U_i",par_gate="rx"):
    pii=np.pi
    if th_i[0]==None:
        th_i=np.round(np.random.rand(n)*2*pii,3)
        
    qc=QuantumCircuit(n)
    for i in range(n):
        if par_gate=="rz":
            qc.rz(th_i[i],i)
        elif par_gate=="rx":
            qc.rx(th_i[i],i)
        elif par_gate=="ry":
            qc.ry(th_i[i],i)
        elif par_gate=="u1":
            qc.u1(th_i[i],i)
        else:
            raise Exception("the gate should be paramaterized by single parameter!")
            
    qc.name = block_name
    return qc

In [11]:
#example
even_block().draw()

┌───────────┐
q_0: ┤ RX(4.162) ├
     ├───────────┤
q_1: ┤ RX(2.777) ├
     └┬──────────┤
q_2: ─┤ RX(1.55) ├
     ┌┴──────────┤
q_3: ┤ RX(5.045) ├
     └───────────┘

Function to add a layer/s

In [12]:
def add_layers(qc, L=1 , pair_blocks=["rz","rx"], th_in=[[None]]):
    n=qc.num_qubits
    if th_in[0][0]==None:
        th_in= np.round(np.random.rand(2*L,n)*2*pii,3)
    else:
        L=int(len(th_in)/2)
        n=int(len(th_in[0]))
        
    for i in range(L):
        odd_name="U_{}".format(i*2+1)
        even_name="U_{}".format((i+1)*2)
        odd = odd_block(n,th_i=th_in[i*2],block_name=odd_name,par_gate=pair_blocks[0])
        even= even_block(n,th_i=th_in[(i*2)+1],block_name=even_name,par_gate=pair_blocks[1])
        
        qc.append(odd,range(n))
        qc.append(even,range(n))
        qc.barrier()

In [13]:
qc=QuantumCircuit(4)
add_layers(qc,th_in=[[1,2,2,1],[1,2,1,1]])
qc.draw()

┌──────┐┌──────┐ ░ 
q_0: ┤0     ├┤0     ├─░─
     │      ││      │ ░ 
q_1: ┤1     ├┤1     ├─░─
     │  U_1 ││  U_2 │ ░ 
q_2: ┤2     ├┤2     ├─░─
     │      ││      │ ░ 
q_3: ┤3     ├┤3     ├─░─
     └──────┘└──────┘ ░

extracting the resulting state:

In [14]:
def psi_th(th_in):
    n=int(len(th_in[0]))
    qc=QuantumCircuit(n)
    add_layers(qc, pair_blocks=["rz","rx"], th_in=th_in)
    
    # Let's see the result
    backend = Aer.get_backend('statevector_simulator')
    final_state = execute(qc,backend).result().get_statevector()
    return final_state

In [15]:
#example
final_state=psi_th([[1,2,2,1],[1,2,1,1]])
from qiskit_textbook.tools import array_to_latex
array_to_latex(final_state, pretext="\\text{Statevector} = ")

<IPython.core.display.Math object>

a function that return the norm^2 of the difference between the final state and some other random state.

In [16]:
def eps(unzipped_th_in):  #the function to be minimized
    n=4
    th_in = [unzipped_th_in[x:x+n] for x in range(0, len(unzipped_th_in), n)]
    psi=psi_th(th_in)
    phi=random_statevector(2**n).data  #define it out of this function for security
    dif=np.array(psi)-np.array(phi)
    epslon= np.dot(np.transpose(np.conjugate(dif)),dif)
    return np.real(epslon)

In [17]:
#example
eps([1,2,4,1,1,2,1,1])

2.012555843682804

Finally the minimum set of all parameters that minimize that difference (2 layer example)

In [23]:
pii=np.pi
x0=np.random.rand(4*2) #initial random guess
result=optimize.minimize(eps,x0,bounds=((0,2*pii),)*4*2)

In [24]:
result

      fun: 1.726979441265252
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 17352972.37437514,  24570352.35719074,  42745698.30457224,
        40649273.38976165,  60717163.20562776,  73128346.46986409,
       -21468012.53320691,  77862039.36830974])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 432
      nit: 4
   status: 0
  success: True
        x: array([0.55854112, 0.05673774, 0.954339  , 0.46276059, 0.8685632 ,
       0.65788789, 0.53107954, 0.92584716])

Now, using these function we can simply generate a graph between the number of layers and the minimum difference reached.